In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

In [2]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [55]:
len(df)

523774

In [56]:
df.activity_id.unique()

array(['HighResMIP', 'CMIP', 'CFMIP', 'ScenarioMIP', 'AerChemMIP',
       'RFMIP', 'FAFMIP', 'DAMIP', 'LUMIP', 'CDRMIP', 'GMMIP', 'C4MIP',
       'OMIP', 'PMIP', 'LS3MIP', 'DCPP', 'PAMIP', 'ISMIP6'], dtype=object)

In [57]:
# Examples of institutional ids
df.institution_id.unique()

array(['CMCC', 'EC-Earth-Consortium', 'MOHC', 'ECMWF', 'NOAA-GFDL',
       'IPSL', 'E3SM-Project', 'CNRM-CERFACS', 'NASA-GISS', 'BCC',
       'MIROC', 'AWI', 'NCAR', 'MRI', 'CCCma', 'SNU', 'CAS', 'INM',
       'NUIST', 'HAMMOZ-Consortium', 'CAMS', 'MPI-M', 'DKRZ', 'DWD', 'UA',
       'NIMS-KMA', 'NCC', 'FIO-QLNM', 'KIOST', 'CSIRO-ARCCSS', 'CSIRO',
       'CCCR-IITM', 'THU', 'AS-RCEC', 'NERC', 'RUBISCO'], dtype=object)

CMIP6 experiment_ids <br>
`'amip', 'historical', 'piControl', '1pctCO2', 'abrupt-4xCO2', 'esm-piControl', 'esm-hist'`


CMIP6 variables https://docs.google.com/spreadsheets/d/1UUtoz6Ofyjlpx5LdqhKcwHFz2SGoTQV2_yekHyMfL9Y/edit?gid=1221485271#gid=1221485271 <br>
`tos` - Sea Surface Temperature [°C]  <br>
`tas` - Near-Surface Air Temperature [K]

Table_id (column) https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q/edit  <br>
`Amon` - Monthly atmospheric data  <br>
`day`  - Daily Data (extension - contains both atmospheric and oceanographic data) <br>
`3hr`  - Atmosphere sampled every 3 hours

In [6]:
'''
Selecting
    activity_id:    CMIP
    institution_id: NCAR, AWI  (?)
    experiment_id:  historical  (?)
    table_id:       Amon   (?)
    variable_id:    tas
    
    member_id: ?
    source_id: ? AWI-ESM-1-1-LR (AWI)
'''

cmip = df.loc[(df.activity_id == 'CMIP') & 
              (df.institution_id == 'AWI') & 
              (df.source_id == 'AWI-ESM-1-1-LR') &
              (df.experiment_id == 'historical') &
              # (df.member_id == 'r1i1p1f1') &
              (df.table_id == 'Amon') &
              (df.variable_id == 'tas') 
            ]
cmip

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
416230,CMIP,AWI,AWI-ESM-1-1-LR,historical,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/histo...,NaN,20200212


In [3]:
#!/usr/bin/env python
from __future__ import print_function
import requests
import xml.etree.ElementTree as ET
import numpy

# Author: Unknown
# I got the original version from a word document published by ESGF
# https://docs.google.com/document/d/1pxz1Kd3JHfFp8vR2JCVBfApbsHmbUQQstifhGNdc6U0/edit?usp=sharing

# API AT: https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API#results-pagination

In [4]:
def esgf_search(server="https://esgf-node.llnl.gov/esg-search/search",
                files_type="OPENDAP", local_node=True, project="CMIP6",
                verbose=False, format="application%2Fsolr%2Bjson",
                use_csrf=False, **search):
    client = requests.session()
    payload = search
    payload["project"] = project
    payload["type"]= "File"
    if local_node:
        payload["distrib"] = "false"
    if use_csrf:
        client.get(server)
        if 'csrftoken' in client.cookies:
            # Django 1.6 and up
            csrftoken = client.cookies['csrftoken']
        else:
            # older versions
            csrftoken = client.cookies['csrf']
        payload["csrfmiddlewaretoken"] = csrftoken

    payload["format"] = format

    offset = 0
    numFound = 10000
    all_files = []
    files_type = files_type.upper()
    while offset < numFound:
        payload["offset"] = offset
        url_keys = [] 
        for k in payload:
            url_keys += ["{}={}".format(k, payload[k])]

        url = "{}/?{}".format(server, "&".join(url_keys))
        print(url)
        r = client.get(url)
        r.raise_for_status()
        resp = r.json()["response"]
        numFound = int(resp["numFound"])
        resp = resp["docs"]
        offset += len(resp)
        for d in resp:
            if verbose:
                for k in d:
                    print("{}: {}".format(k,d[k]))
            url = d["url"]
            for f in d["url"]:
                sp = f.split("|")
                if sp[-1] == files_type:
                    all_files.append(sp[0].split(".html")[0])
    return sorted(all_files)

In [10]:
result = esgf_search(activity_id='CMIP', 
                     table_id='Amon',
                     variable_id='tas', 
                     experiment_id='historical',
                     institution_id='NCAR', 
                     source_id="CESM2-WACCM",
                     member_id="r1i1p1f1")
result

https://esgf-node.llnl.gov/esg-search/search/?activity_id=CMIP&table_id=Amon&variable_id=tas&experiment_id=historical&institution_id=NCAR&source_id=CESM2-WACCM&member_id=r1i1p1f1&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0


['http://aims3.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/NCAR/CESM2-WACCM/historical/r1i1p1f1/Amon/tas/gn/v20190227/tas_Amon_CESM2-WACCM_historical_r1i1p1f1_gn_185001-201412.nc',
 'http://esgf-data.ucar.edu/thredds/dodsC/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/historical/r1i1p1f1/Amon/tas/gn/v20190227/tas_Amon_CESM2-WACCM_historical_r1i1p1f1_gn_185001-201412.nc',
 'http://esgf-data04.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/historical/r1i1p1f1/Amon/tas/gn/v20190227/tas_Amon_CESM2-WACCM_historical_r1i1p1f1_gn_185001-201412.nc']

In [11]:
# there are mulitple sources of the same data--need to pick one
files_to_open = result[-1:]

ds = xr.open_mfdataset(files_to_open, combine='by_coords')
ds

/home/.pixi/envs/default/lib/python3.12/site-packages/xarray/conventions.py:440: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset> Size: 438MB
Dimensions:    (lat: 192, lon: 288, time: 1980, nbnd: 2)
Coordinates:
  * lat        (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * time       (time) object 16kB 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
Dimensions without coordinates: nbnd
Data variables:
    time_bnds  (time, nbnd) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat_bnds   (lat, nbnd) float32 2kB dask.array<chunksize=(192, 2), meta=np.ndarray>
    lon_bnds   (lon, nbnd) float32 2kB dask.array<chunksize=(288, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 438MB dask.array<chunksize=(1980, 192, 288), meta=np.ndarray>
Attributes: (12/46)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     CMIP
    case_id:                         4
    cesm_casename:                   b.e21.BWHIST.f09_g17.CMIP6-historical-WA...
    contact:                         cesm_cmip6@ucar.edu
    creation_date:                   2019-01-30T22:37:15Z
    ...                              ...
    variant_info:                    CMIP6 CESM2 hindcast (1850-2014) with hi...
    variant_label:                   r1i1p1f1
    branch_time_in_parent:           20075.0
    branch_time_in_child:            674885.0
    branch_method:                   standard
    DODS_EXTRA.Unlimited_Dimension:  time